In [9]:
import pandas as pd
import  numpy as np

data = pd.read_csv("./data/2_preprcossing_train.csv", index_col='id')
data.head(5)
X = data.iloc[:, :-1]
y = data['y']
y

id
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
41183    1.0
41184    0.0
41185    0.0
41186    1.0
41187    0.0
Name: y, Length: 41188, dtype: float64

In [10]:
from sklearn.model_selection import train_test_split
random_seed = 42
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=random_seed, stratify=y)


In [11]:
from sklearn.impute import SimpleImputer
impute = SimpleImputer(strategy='most_frequent')
X_train = impute.fit_transform(X_train)
X_valid = impute.transform(X_valid)
X_train = pd.DataFrame(X_train, columns=X.columns)

In [12]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
oversample = BorderlineSMOTE()
X_over, y_over = oversample.fit_resample(X_train, y_train)
y_over.value_counts()

0.0    27411
1.0    27411
Name: y, dtype: int64

In [25]:
from lightgbm.sklearn import LGBMClassifier
params = {
          'max_depth': [3,4,5,6,7,8,9,10],  # 最大深度
            # 树的数量
          'learning_rate': [0.01, 0.02, 0.05, 0.1,0.2,0.3],
          'num_leaves': [15, 31, 49, 63, 127],
          'min_child_samples': [100, 200, 300, 400, 500]
          }
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
lgb = LGBMClassifier(reg_lambda=0.01,
                     max_depth=6,
                     num_leaves=15,
                     min_child_samples=400,
                     boosting_type='dart',
                     n_estimators = 200)
random_search = HalvingRandomSearchCV(estimator=lgb, param_distributions=params,
                                      factor=2, resource='n_estimators',
                                      scoring='f1'
                                      max_resources=30,
    random_state=random_seed)
random_search.fit(X_train, y_train)

HalvingRandomSearchCV(estimator=LGBMClassifier(boosting_type='dart',
                                               max_depth=6,
                                               min_child_samples=400,
                                               n_estimators=200, num_leaves=15,
                                               reg_lambda=0.01),
                      factor=2, max_resources=30,
                      param_distributions={'learning_rate': [0.01, 0.02, 0.05,
                                                             0.1, 0.2, 0.3],
                                           'max_depth': [3, 4, 5, 6, 7, 8, 9,
                                                         10],
                                           'min_child_samples': [100, 200, 300,
                                                                 400, 500],
                                           'num_leaves': [15, 31, 49, 63, 127]},
                      random_state=42, resource='n_estimators')

In [27]:
y_train_pred = random_search.predict(X_over)
y_valid_pred = random_search.predict(X_valid)

from sklearn.metrics import f1_score
print(f1_score(y_over, y_train_pred))
print(f1_score(y_valid, y_valid_pred))

0.9513112758658847
0.5505356311131813


In [28]:
random_search.best_params_

{'num_leaves': 63,
 'min_child_samples': 100,
 'max_depth': 10,
 'learning_rate': 0.3,
 'n_estimators': 16}

In [29]:
lgb.get_params()

{'boosting_type': 'dart',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_samples': 400,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 200,
 'n_jobs': -1,
 'num_leaves': 15,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.01,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}